In [14]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [15]:
def pretty_print_docs(docs):
    for i, d in enumerate(docs):
        print(f"문서 {i+1}:\n\n" + d.page_content)
        print(f"{'-' * 100}\n")

In [16]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

from langchain_teddynote.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever

from langchain_openai import ChatOpenAI

## **맥락적 압축(ContextualCompression)**

`LLMChainExtractor` 를 활용하여 생성한 `DocumentCompressor` 를 `retriever` 에 적용한 것이 바로 `ContextualCompressionRetriever` 입니다.

In [17]:
loader = TextLoader("./data/appendix-keywords.txt", encoding='utf-8')

In [18]:
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = loader.load_and_split(text_splitter)

In [19]:
embeddings = OpenAIEmbeddings(api_key=key)

In [20]:
# 기본 retriever 생성
retriever = FAISS.from_documents(texts, embedding=embeddings).as_retriever()

In [21]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

In [22]:
# LLM을 사용하여 문서 압축기 생성
compressor = LLMChainExtractor.from_llm(llm)

In [23]:
# 문서 압축기와 리트리버를 사용하여 컨텍스트 압축 리트리버 생성
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,         # LLM을 사용하여 만든 문서 압축기 지정
    base_retriever=retriever,           # 위에서 만든 기본 retriever를 지정
)

In [24]:
# 컨텍스트 압축 리트리버를 사용하여 관련 문서 검색
compressed_docs = (
    compression_retriever.invoke( 
        "Semantic Search 에 대해서 알려줘."
    )
)

In [25]:
pretty_print_docs(compressed_docs)      # 질문과 관련된 문서만 나온다.

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
----------------------------------------------------------------------------------------------------

